In [ ]:
%%javascript
$('#appmode-leave').hide();
$('#copy-binder-link').hide();
$('#visit-repo-link').hide();

# [Open empty notebook](empty.ipynb)

# Crystal Violet Virtual Experiment

In [ ]:
import ipywidgets as ipw
import json
import random
import time
import pandas as pd
import os
import webbrowser
import math
import numpy as np
from IPython.display import display, Markdown, FileLink, clear_output

In [ ]:
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
   return false;
}

In [ ]:
with open(".lab.json") as infile:
    jsdata = json.load(infile)

params = jsdata["cv"]

In [ ]:
t = int( time.time() * 1000.0 )
random.seed( ((t & 0xff000000) >> 24) +
             ((t & 0x00ff0000) >>  8) +
             ((t & 0x0000ff00) <<  8) +
             ((t & 0x000000ff) << 24)   )

params["NaOH"] = 0.5 * random.gauss(1,params["error"])
params["CV"] = 2.5e-5 * random.gauss(1,params["error"])

In [ ]:
def run_experiment():

    tt = random.gauss(T.value,params["et"])
    x0 = v0.value + random.gauss(0,params["ev"])
    x1 = v1.value + random.gauss(0,params["ev"])
    x2 = v2.value + random.gauss(0,params["ev"])

    lnk = params["A"] - params["B"]/params["R"]/(tt+273.15)
    kr = math.exp(lnk)
    
    vt = x0 + x1 + x2
    coh = params["NaOH"] * x0 / vt
    ccv = params["CV"] * x1 / vt
    
    kd = kr * math.pow(coh,params["beta"])

    Abs = params["A0"]*ccv
        
    res = pd.DataFrame(columns=["#Time [s]" , "Absorbance"])
    for i in range(2, params["nTime"]):
        var_list = []
        var_list.append(i)
            
        expVal = Abs * math.exp(-kr*i) + abs(random.gauss(0,params["error"]) + 0.008)
        var_list.append(expVal)
        res.loc[len(res)] = var_list
    
    res.to_csv(respath.value, index=False)
    local_file = FileLink(respath.value, result_html_prefix="Click here to download: ")
    with out_P:
        display(local_file)
        display(res.tail(params["nTime"]))


In [ ]:
out_Error = ipw.Output()
out_P = ipw.Output()

# output filename
fileName = "results.csv"
respath = ipw.Text(fileName)

v0 = ipw.FloatSlider(value=10, min=0, max=20)
v1 = ipw.FloatSlider(value=10, min=0, max=20)
v2 = ipw.FloatSlider(value=10, min=0, max=20)
T  = ipw.FloatSlider(value=25, min=10, max=40)

def reset(btn):
    if os.path.exists(respath.value):
        os.remove(respath.value)
    
    with out_Error:
        out_Error.clear_output()
        
    with out_P:
        out_P.clear_output()
        
    clear_output()
    create_ipw()

def calc(btn):
    out_P.clear_output()
    run_experiment()

# interactive buttons ---
btn_calc = ipw.Button(description="Perform Experiment", layout=ipw.Layout(width="150px"))
btn_calc.on_click(calc)

btn_reset = ipw.Button(description="Reset Experiment", layout=ipw.Layout(width="150px"))
btn_reset.on_click(reset)

# -- output widgets
def create_ipw():
    rows = []

    label_layout = ipw.Layout(width='300px')

    rows.append(ipw.HBox([ipw.Label('Output filename : ',layout=label_layout),respath]))
    rows.append(ipw.HBox([ipw.Label(value="2.5x10$^{-5}$M stock solution of CV (ml)",layout=label_layout),v0]))
    rows.append(ipw.HBox([ipw.Label(value="0.5 M stock solution of NaOH (ml)",layout=label_layout),v1]))
    rows.append(ipw.HBox([ipw.Label(value="Deionised water (ml)",layout=label_layout),v2]))
    rows.append(ipw.HBox([ipw.Label(value="Temperature ($^\circ$C)",layout=label_layout),T]))

    rows.append(ipw.HBox([btn_reset,btn_calc]))
    rows.append(ipw.HBox([out_Error]))
    rows.append(ipw.HBox([out_P]))
    display(ipw.VBox(rows))

create_ipw()
